In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
# you can find the latest package @ https://github.com/kubeflow/pipelines/releases
#KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz'
#!pip3 install $KFP_PACKAGE --upgrade

In [10]:
EXPERIMENT_NAME = 'Hellow world!'
GCLOUD_SDK = 'google/cloud-sdk:latest'

# Managing secrets 

This section demonstrates how you access Kubernetes secrets using Kubeflow Pipeline SDK.

Reference documentation: 
* https://www.kubeflow.org/docs/gke/gcp-e2e/#check-the-permissions-for-your-training-component 
* https://cloud.google.com/kubernetes-engine/docs/concepts/secret 
* https://kubernetes.io/docs/concepts/configuration/secret/

Specifically note that you can view, or add secrets to your K8 cluster using:

```console
kubectl get secrets
kubectl create secret generic db-user-pass --from-file=./username.txt --from-file=./password.txt
```

In [1]:
import kfp
import kfp.dsl as dsl
from kfp.gcp import use_gcp_secret
from kubernetes import client as k8s_client
from kfp import compiler
from kfp import notebook
from kfp import components as comp

### google cloud Auth methods
To retrieve a GCP secret from K8 you can use the container operation 
```console
.apply(use_gcp_secret('secret name'))
```
Application Default Credentials (ADC) provide a method to get credentials used in calling Google APIs.However to add the secret to user enviroment to use for applications such as Gsutil you need to run the following 
``` console
gcloud auth activate-service-account --key-file /secret/gcp-credentials/user-gcp-sa.json'''
```
The gcloud auth application-default command group allows you to manage active credentials on your machine that are used for local application development.


In [4]:
def using_secrets_op():
    return dsl.ContainerOp(
        name = 'Using Secrets',
        image = GCLOUD_SDK,
        command=['sh', '-c'],
        arguments = [  'gcloud auth activate-service-account --key-file /secret/gcp-credentials/user-gcp-sa.json']
    ).apply(use_gcp_secret('user-gcp-sa'))


#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [8]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='secrets',
   description='A toy pipeline demonstrates use of gcp secretes'
)
def calc_pipeline(
):
    #Creating a one step pipeline
    using_secrets = using_secrets_op() #Returns a dsl.ContainerOp class instance. 

#### Compile the pipeline

In [11]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [12]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [13]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working

/opt/conda/lib/python3.6/site-packages/kfp/_client.py:194: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)
